In [ ]:
#installing a package
#!pip3 install <package name> 

In [ ]:
# 2. remove jiwani hard coding apply any condition so that jiwani will be selected automatically...........done
# get islamic month and year from internet and put as an input
# 3. Make a method to process all files according to their Islamic Month name and Islamic date automatically
# 4. Calculate Parameters and recheck its accuracy with data 
# 5. Make GUI interface
# 6. Make it customizable code and reusable code apply OOP

In [1]:
import pandas as pd
import math 
import os
from fpdf import FPDF
import webbrowser
from datetime import datetime
import requests

In [36]:
import os
import math
import pandas as pd
from datetime import datetime
import webbrowser

class MoonCalc:
    def __init__(self, file_path, date, Month, year, dst):
        self.path = file_path.replace('"', '')
        self.date = date.replace('"', '')
        self.month = Month.replace('"', '')
        self.year = year.replace('"', '')
        self.dst = dst.replace('"', '') if dst else None

    def data(self, *args):
        def set_axis(df):
            columns = ["year", "h", "cd", "conj", "f", "wk", "mon", "day", "set", "Saz", "age",
                       "Alt", "Maz", "dz", "Mag", "El", "mset", "lag", "best", "cat"]
            df.columns = columns
            return df

        def illum(a):
            return round(50 * (1 - math.cos(math.radians(a))), 1)

        try:
            df = pd.read_fwf(os.path.join(args[0], args[1]))
        except Exception as e:
            print(f"Error loading file {args[1]}: {e}")
            return pd.DataFrame()

        df = set_axis(df)
        dfa = df.drop(['f', 'Mag', 'wk'], axis=1)
        dfs = dfa.loc[:, :'conj']
        dfd = dfa.loc[:, 'mon':'cat']

        dfd = dfd.bfill()
        dfs = dfs.ffill()

        dfg = dfs.combine_first(dfd)
        dfg.drop_duplicates(inplace=True)
        dfg.dropna(inplace=True)

        for x in ['conj', 'set', 'mset', 'best']:
            dfg[x] = dfg[x].replace(' ', ':', regex=True)

        for x in ['cd', 'day', 'year', 'lag', 'Alt', 'Saz', 'dz', 'Maz']:
            dfg[x] = dfg[x].astype(int).astype(str)

        dfg['mon'] = dfg['mon'].str[:3]
        dfg['h'] = dfg['h'].str[:3]
        dfg['date'] = pd.to_datetime(dfg['day'] + dfg['mon'] + dfg['year'], format='%d%b%Y')
        dfg.set_index('date', inplace=True)

        dfg['conj_time'] = pd.to_datetime(dfg['cd'] + dfg['h'] + dfg['year'] + ' ' + dfg['conj'])

        dfg['Station'] = args[1].split('.', 1)[0] if args else self.loc

        dfg['El'] = dfg['El'].astype(int)
        dfg['ilum'] = dfg['El'].apply(illum)

        return dfg

    def sort(self, *args):
        df = self.data(args[0], args[1])
        if df.empty:
            return pd.DataFrame()

        df = df[['Station', 'set', 'lag', 'Alt', 'Saz', 'dz', 'El', 'ilum', 'cat', 'age', 'conj_time']]
        df['Station'] = df['Station'].astype("category").cat.set_categories(sorted(df['Station'].unique()))
        return df

    def all_files(self):
        df = pd.DataFrame()
        if os.path.exists(self.path):
            for root, dirs, files in os.walk(self.path):
                for filename in files:
                    d = self.sort(root, filename)
                    if not d.empty:
                        df = pd.concat([df, d], ignore_index=False)
            return df
        else:
            print("Directory does not exist")
            return df

    def calculate(self):
        dfs = self.all_files()
        dfd = pd.DataFrame()
        if not dfs.empty:
            dfs['date'] = dfs.index
            dfs.set_index('date', inplace=True)
            dfs.index = pd.to_datetime(dfs.index)

            target_date = pd.to_datetime(self.date)
            if target_date in dfs.index:
                daily_df = dfs.loc[target_date]
                daily_df = daily_df.sort_values('Station')

                dfd = daily_df.loc[:, :'cat']
                dfd['Station'] = dfd['Station'].astype(str) + " (" + dfd['set'].astype(str) + ")"
                dfd.drop(columns="set", inplace=True)
                

                dfd.rename(columns={
                    'Station': 'STATION(Sunset)',
                    'lag': 'LAG TIME(Minutes)',
                    'Alt': 'MOON ALTITUDE(Degrees)',
                    'Saz': 'SUN_AZIMUTH(Degrees)',
                    'dz': 'DAZ(Degrees)',
                    'El': 'ELONGATION(Degrees)',
                    'ilum': 'ILLUMINATION(%)',
                    'cat': 'CRITERION'
                }, inplace=True)

        return dfd

    def Select_city(self):
        dfs = self.all_files()
        if dfs.empty:
            return {}

        dfs.index = pd.to_datetime(dfs.index)
        target_date = pd.to_datetime(self.date)

        if target_date not in dfs.index:
            print("Max df is not selected")
            return {}

        df_today = dfs.loc[target_date].sort_values("Station")
        df_today = df_today[df_today["set"] == df_today["set"].max()]

        return {
            "age": df_today.age.values[0].split(" "),
            "dt": df_today.conj_time.dt.strftime("%d-%m-%Y").values[0],
            "tm": df_today.conj_time.dt.strftime("%H:%M:%S").values[0],
            "city": df_today.Station.values[0]
        }
    
    def pdf(self):
        Format = "Arial"        
        data = {'Station':'  STATION    (Sunset)','lag':'LAG TIME  (Min)','Alt':'MOON ALTITUDE   (Deg)', 
                                      'Saz':'SUN_AZIMUTH (Deg)',
                                      'dz':'DAZ   (Deg)  ',
                                      'El':'ELONGATION  (Deg)','ilum':'ILLUMINATION  (%)',
                                      'cat':'CRITERION   '}
        df = pd.DataFrame()
        df = self.calculate()
        if df.empty == True: return print("Date not Found") 
        Date = datetime.strptime(self.date,"%Y-%m-%d")
        Date =Date.strftime("%d-%m-%Y")
        pdf = PDF(self.path, self.date, self.month, self.year , self.dst ,'L', 'mm', 'A4')
        pdf.add_page()
        pdf.set_font(Format,'B',11)
        li = []
        for x in data.values():li.append(x)
        width = [40,30,38,33,22,31,33,26,40,40]
        start = 25
        pdf.x = start
        offset = pdf.x + width[0]
        sx = pdf.x
        i = 0
        top = 40
        pdf.y = top
        for head in li:    
            pdf.multi_cell(width[i],7,head,border = 1,align = "C")
            # Reset y coordinate
            pdf.y = top
            # Move to computed offset    
            pdf.x = offset
            i += 1
            offset = offset+ width[i]
        h = pdf.font_size * 2.5
        pdf.y = 54
        pdf.set_font(Format,'',11)
        for index, row in df.iterrows():
            i = 0
            pdf.x = start
            for data in row.values:
                pdf.cell(width[i], h, str(data),border = 1,align='C') # write each data for the row in its cell
                i +=1  
            pdf.ln()      
        ls = ["(A)  Easily visible",
                         "(B) Visible under perfect conditions",
                         "(C)  May need optical aid to find the crescent Moon",
                        "(D)  Will need optical aid to find the crescent Moon",
                        "(E)  Not visible with a telescope",
                        "(F)  Not visible, below the Danjon limit"]
        
        pdf.ln()
        pdf.set_font(Format, 'BU', 12)
        h = 5
        pdf.ln()
        pdf.set_font(Format, '', 11)
        if self.dst:
            pdf.output(self.dst+"\\"+Date+".pdf",'F')
            webbrowser.open_new(self.dst+"\\"+Date+'.pdf')
        else:
            pdf.output(Date+'.pdf','F') # save pdf
            webbrowser.open_new(Date+'.pdf') # open pdf in browser  


In [37]:
class PDF(FPDF, MoonCalc):
    def __init__(self, file_path, date, Month, year, dst, *args, **kwargs):
        # Initialize FPDF
        FPDF.__init__(self, *args, **kwargs)  
        
        # Initialize MoonCalc with required parameters
        MoonCalc.__init__(self, file_path, date, Month, year, dst)

    
    def footer(self):
        self.set_y(-27)
        Format = "Arial"
        
        # Visibility Criterion
        ls = [
            "(A)  Easily visible",
            "(B)  Visible under perfect conditions",
            "(C)  May need optical aid to find the crescent Moon",
            "(D)  Will need optical aid to find the crescent Moon",
            "(E)  Not visible with a telescope",
            "(F)  Not visible, below the Danjon limit"
        ]

        self.set_font(Format, 'BU', 12)
        self.cell(297, 5, "Visibility Criterion:", ln=1, align='L')
        self.ln(2)

        self.set_font(Format, '', 11)
        sp = "  "
        self.multi_cell(280, 5, txt=sp.join(ls), align='L')
        
        self.set_font(Format, 'I', 8)
        self.cell(270, 10, 'Computer Generated', 0, 0, 'R')

        self.ln(5)

    def header(self):
        data = self.Select_city()
        age = data["age"]
        dt = data["dt"]
        tm = data["tm"]
        city = data["city"]

        Format = "Arial"        

        Date = datetime.strptime(self.date, "%Y-%m-%d")
        Date = Date.strftime("%d-%m-%Y")

        self.set_font(Format, 'B', 16)
        h = 7
        w = 297
        self.cell(w, h, txt="PARAMETERS OF THE NEW MOON " + self.month + " " + self.year, ln=1, align='C')
        self.cell(w, h, txt="AT THE TIME OF SUNSET ON " + Date, ln=1, align='C')
        self.cell(w, h, txt=f"(Conjunction on {dt} {tm} PST) ", ln=1, align='C')
        self.cell(w, h, txt = f"Moon Age at the time of Sunset on {Date} ({city}): {age[0]} hrs {age[1]} mins",ln = 1, align = 'C')
        self.ln()



In [38]:
def get_hijri_date(date):
    """
    Convert a Gregorian date (YYYY-MM-DD) to an Islamic (Hijri) date and get the next Islamic month.
    
    :param georgian_date: str, Gregorian date in 'YYYY-MM-DD' format
    :return: dict, Hijri date details including next Islamic month
    """
    try:
        # Convert date from YYYY-MM-DD to DD-MM-YYYY for API
        converted_date =date
        
        # API URL with formatted date
        url = f"https://api.aladhan.com/v1/gToH/{converted_date}?calendarMethod=UAQ"
        
        # Send GET request
        response = requests.get(url)
        
        # Check if request was successful
        if response.status_code == 200:
            data = response.json()  # Convert response to JSON
            
            # Extract Hijri date details
            hijri_date = data["data"]["hijri"]["date"]  # Example: "28-08-1446"
            hijri_month = data["data"]["hijri"]["month"]["en"].upper()  # Convert to uppercase
            hijri_day = data["data"]["hijri"]["day"]  # Example: "28"
            hijri_year = int(hijri_date.split('-')[2])  # Convert year to integer
            
            # Get current Hijri month number
            hijri_month_number = data["data"]["hijri"]["month"]["number"]  # Example: 8 for Sha'ban
            
            # List of Islamic months in uppercase
            ISLAMIC_MONTHS = [
                "MUHARRAM", "SAFAR", "RABI UL AWWAL", "RABI US SANI",
                "JUMADI UL-AWWAL", "JUMADI US SANI", "RAJAB", "SHABAN",
                "RAMADAN", "SHAWWAL", "ZUL-QADDAH", "ZUL-HIJJAH"
            ]
            
            # Calculate next Islamic month
            next_hijri_month = ISLAMIC_MONTHS[hijri_month_number % 12]  # Modulo to cycle months
            
            # If next month is MUHARRAM, increment the Hijri year
            if next_hijri_month == "MUHARRAM":
                hijri_year += 1
            
            return {
                "hijri_day": hijri_day,
                "hijri_month": next_hijri_month,
                "hijri_year": str(hijri_year)
            }
        else:
            return {"error": "Failed to fetch Hijri date"}
    
    except Exception as e:
        return {"error": str(e)}

In [39]:
def generate_pdf(path, date, dst):
#     path = "D:/code/Moon_predictor/Data_2035"
#     date = "28-02-2025"
#     dst = 'D:/Output'
    hijri_data = get_hijri_date(date)
    month = hijri_data['hijri_month']
    year = hijri_data['hijri_year']
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
    Moon.pdf()
generate_pdf("Data_2035","28-02-2025",'Output')


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Specify the model name from Hugging Face
model_name = "mistral-7b"  # Change this if you're using a different Mistral model

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("Model and tokenizer downloaded successfully.")

OSError: mistral-7b is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [51]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "C:\\Users\Administrator\.ollama\models"  # Update this path to where your model is saved locally
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
    

def parse_command_with_mistral(command_str):
    """
    This function will send the command to Mistral to extract input params.
    It expects a string like: 'Generate pdf for input dir: Data_2035, Date: 28-02-2025, Output dir: Output'
    and return parsed parameters as a tuple (input_dir, date, output_dir).
    """
    
    # Create the prompt for Mistral to understand
    prompt = f"Extract the input parameters from the following command:\n{command_str}\n"
    
    # Encode the prompt and send to the model
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=150)

    # Decode the model's response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Now, parse the generated text to extract the parameters
    # Example expected output: "Data_2035, 28-02-2025, Output"
    parsed_params = generated_text.split(",")  # Assuming the response is comma-separated
    
    if len(parsed_params) == 3:
        input_dir = parsed_params[0].strip()
        date = parsed_params[1].strip()
        output_dir = parsed_params[2].strip()
        return input_dir, date, output_dir
    else:
        raise ValueError("Parsing failed. Expected format: 'input_dir, date, output_dir'")

def generate_pdf(input_dir, date, output_dir):
    """
    This is the function that processes the input params and generates the PDF.
    For demonstration purposes, it's just printing out the values.
    """
    print(f"Generating PDF for:\nInput Directory: {input_dir}\nDate: {date}\nOutput Directory: {output_dir}")

# Example usage
command = "Generate pdf for input dir: Data_2035, Date: 28-02-2025, Output dir: Output"
input_dir, date, output_dir = parse_command_with_mistral(command)

# Now pass the parsed params to the generate_pdf function
generate_pdf(input_dir, date, output_dir)


<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14024\1718431733.py:3: SyntaxWarning: invalid escape sequence '\A'
  model_path = "C:\\Users\Administrator\.ollama\models"  # Update this path to where your model is saved locally
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14024\1718431733.py:3: SyntaxWarning: invalid escape sequence '\A'
  model_path = "C:\\Users\Administrator\.ollama\models"  # Update this path to where your model is saved locally


OSError: Can't load tokenizer for 'C:\Users\Administrator\.ollama\models'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'C:\Users\Administrator\.ollama\models' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
prompt = build_prompt()
params = generate_inputs(prompt)

if params:
    generate_pdf(params['path'], params['date'], params['dst'])
